In [1]:
# import and install the required libraries
import pandas as pd
import numpy as np
!conda install -c conda-forge geocoder --yes
import geocoder

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181013081632-0000
Solving environment: done

## Package Plan ##

  environment location: /opt/ibm/conda/miniconda3

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.5.11               |           py35_0         636 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    orderedset-2.0             |           py35_0         685 KB  conda-forge
    ca-certificates-2018.8.24  |       ha4d7672_0         136 KB  conda-forge
    ratelim-0.1.6              |           py35_0           5 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    ----------------------------------------------------------

In [35]:
#import all the libraries that is needed
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

weblink = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#to get the content of the page using request comand
page = requests.get(weblink).text
soup = BeautifulSoup(page, 'lxml')
tblrec = soup.find('tbody')
dc_toronto = {'PostalCode':[],
                'Borough':[],
               'Neighborhood':[]}

for tr in tblrec.find_all('tr')[1:]:
    td=tr.find_all('td')
    #print(td)
    #print()
    #store the postal code
    pocode = td[0].text
    
    # process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    #place the values from the list
    if td[2].a:
        neigh = td[2].a.text
        if td[1].a:
            boro = td[1].a.text
        else:
            boro=td[1].text
    else:
        neigh = (' ').join(td[2].text.split())
        if td[1].a:
            boro = td[1].a.text
        else:
            boro=td[1].text
    
    #check if the value of neighbor is "Not assigned" or not 
    #if not assigned then replace with borough
    if neigh == 'Not assigned':
        neigh = boro
        
    # to place the values in for the 1st time into the toronto dataframe
    if len(dc_toronto['PostalCode'])==0:
        dc_toronto['PostalCode'].append(pocode)
        dc_toronto['Neighborhood'].append(neigh)
        dc_toronto['Borough'].append(boro)
        
    #from the 2nd postal code onwards
    else:
        #check if the postal code is same to the previous postal code
        if dc_toronto['PostalCode'][-1] == pocode:
            # do the concatenation with comma
            dc_toronto['Neighborhood'][-1] = neigh + ','+dc_toronto['Neighborhood'][-1] 
        else:
            dc_toronto['PostalCode'].append(pocode)
            dc_toronto['Neighborhood'].append(neigh)
            dc_toronto['Borough'].append(boro)

print("The dictonary is ready")

#convert the dict to dataframe
df=pd.DataFrame.from_dict(dc_toronto)

#remove all the ones which are "Not assigned"
df.replace('Not assigned', np.nan, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

cols=df.columns.tolist()
cols
cols=cols[-1:]+cols[:-1]
df=df[cols]

  Using cached https://files.pythonhosted.org/packages/21/0a/47fdf541c97fd9b6a610cb5fd518175308a7cc60569962e776ac52420387/beautifulsoup4-4.6.3-py3-none-any.whl
Target directory /home/spark/shared/user-libs/python3/beautifulsoup4-4.6.3.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/bs4 already exists. Specify --upgrade to force replacement.
  Using cached https://files.pythonhosted.org/packages/eb/e2/02d18a1b3021b65409dd860f91cf0d68d79900f172bb3cc93cff21c3c951/lxml-4.2.5-cp35-cp35m-manylinux1_x86_64.whl
Target directory /home/spark/shared/user-libs/python3/lxml-4.2.5.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/lxml already exists. Specify --upgrade to force replacement.
  Using cached https://files.pythonhosted.org/packages/a5/62/bbd2be0e7943ec8504b517e62bab011b4946e1258842bc159e5dfde15b96/html5lib-1.0.1-py2.py3-none-any.whl
  Using cached 

In [36]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern,Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill,Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District,Ryerson"


In [37]:
#now start part 2

In [38]:
from geopy.geocoders import Nominatim

In [39]:
#get the locations of the address
for index, row in df.iterrows():
    address_1 = row['Neighborhood'] 
    address_2 = address_1.split(',')[-1]
    address_3 = address_2+","+"Ontario,Canada"
    #print(address_3) #-- It worked


In [40]:
# to add the 2 new cols in orig dataframe
column_names = ['Latitude', 'Longitude'] 
latlong = pd.DataFrame(columns=column_names)

for index, row in df.iterrows():
    try:
        address_1 = row['Neighborhood'] 
        address_2 = address_1.split(',')[-1]
        address = address_2+","+"Ontario,Canada"
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        latlong = latlong.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        pass
    except:
        address_3 = row['Borough']
        address = address_3+","+"Ontario,Canada"
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        latlong = latlong.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        pass

In [41]:
latlong.head()

,Latitude,Longitude
0,43.757846,-79.315975
1,43.732658,-79.311189
2,43.640080,-79.380150
3,43.722778,-79.450933
4,43.659980,-79.390369


In [43]:
df=pd.concat([df,latlong[['Latitude','Longitude']]],axis=1)
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.757846,-79.315975
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.640080,-79.380150
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.722778,-79.450933
4,M7A,Queen's Park,Queen's Park,43.659980,-79.390369
5,M9A,Etobicoke,Islington Avenue,43.626794,-79.516125
6,M1B,Scarborough,"Malvern,Rouge",43.804930,-79.165837
7,M3B,North York,Don Mills North,43.737178,-79.343451
8,M4B,East York,"Parkview Hill,Woodbine Gardens",43.712078,-79.302567
9,M5B,Downtown Toronto,"Garden District,Ryerson",45.579793,-79.508466
